# Google AI chat models (gemini-pro)

Google AI의 `gemini`와 `gemini-vision` 모델뿐만 아니라 다른 생성 모델에 접근하려면 [langchain-google-genai](https://pypi.org/project/langchain-google-genai/) 통합 패키지의 `ChatGoogleGenerativeAI` 클래스를 사용하면 됩니다.


In [ ]:
# !pip install -qU langchain-google-genai

### API KEY 발급받기

- [링크](https://makersuite.google.com/app/apikey?hl=ko) 에서 API KEY를 발급받아주세요.
- 사용자의 Google API 키를 환경 변수 `GOOGLE_API_KEY`로 설정합니다.


In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

langchain_google_genai 패키지에서 ChatGoogleGenerativeAI 클래스를 가져옵니다.

- ChatGoogleGenerativeAI 클래스는 Google의 Generative AI 모델을 사용하여 대화형 AI 시스템을 구현하는 데 사용됩니다.
- 이 클래스를 통해 사용자는 Google의 대화형 AI 모델과 상호 작용할 수 있습니다.
- 모델과의 대화는 채팅 형식으로 이루어지며, 사용자의 입력에 따라 모델이 적절한 응답을 생성합니다.
- ChatGoogleGenerativeAI 클래스는 LangChain 프레임워크와 통합되어 있어, 다른 LangChain 컴포넌트와 함께 사용할 수 있습니다.


지원되는 모델 정보: https://ai.google.dev/gemini-api/docs/models/gemini?hl=ko

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# 프롬프트를 전달하여 결과를 생성합니다.
answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")

# 결과를 출력합니다.
stream_response(answer)

In [ ]:
from langchain_core.prompts import PromptTemplate

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",  # 사용할 모델을 지정합니다.
)

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    "예/아니오 질문에 대답하세요. {question}는 과일입니까?"
)

# 체인을 생성합니다.
chain = prompt | model

# 결과를 출력합니다.
stream_response(chain.stream({"question": "사과"}))

## Safety Settings

Gemini 모델에는 기본 안전 설정(Satety Settings) 이 있지만, 이를 재정의할 수 있습니다.

만약 모델로부터 많은 "Safety Warnings"를 받고 있다면, 모델의 `safety_settings` 속성을 조정해 볼 수 있습니다.

Google의 [Safety Setting Types](https://ai.google.dev/api/python/google/generativeai/types/SafetySettingDict) 문서에서는 사용 가능한 카테고리와 임계값에 대한 열거형 정보를 제공합니다.

이 문서에는 콘텐츠 필터링 및 안전 설정과 관련된 다양한 카테고리와 해당 임계값이 정의되어 있어, 개발자들이 생성형 AI 모델을 활용할 때 적절한 안전 설정을 선택하고 적용하는 데 도움을 줍니다.

이를 통해 개발자들은 모델이 생성하는 콘텐츠의 안전성과 적절성을 보장하고, 사용자에게 유해하거나 부적절한 내용이 노출되는 것을 방지할 수 있습니다.

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    # 사용할 모델을 "gemini-pro"로 지정합니다.
    model="gemini-1.5-pro-latest",
    safety_settings={
        # 위험한 콘텐츠에 대한 차단 임계값을 설정합니다.
        # 이 경우 위험한 콘텐츠를 차단하지 않도록 설정되어 있습니다. (그럼에도 기본적인 차단이 있을 수 있습니다.)
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

## Batch 단위 실행


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    # 사용할 모델을 "gemini-pro"로 지정합니다.
    model="gemini-1.5-pro-latest",
)

results = llm.batch(
    [
        "대한민국의 수도는?",
        "대한민국의 주요 관광지 5곳을 나열하세요",
    ]
)

for res in results:
    # 각 결과의 내용을 출력합니다.
    print(res.content)

## Multimodal 모델

`langchain-teddynote` 에서 구현한 멀티모달 모델에 `gemini-1.5-pro` 모델을 활용하여 이미지를 텍스트로 변환 가능합니다.

In [ ]:
from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

# 객체 생성
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

system_prompt = (
    "당신은 시인입니다. 당신의 임무는 주어진 이미지를 가지고 시를 작성하는 것입니다."
)

user_prompt = "다음의 이미지에 대한 시를 작성해주세요."

# 멀티모달 객체 생성
multimodal_gemini = MultiModal(
    llm, system_prompt=system_prompt, user_prompt=user_prompt
)

In [ ]:
# 샘플 이미지 경로(파일의 경로, URL 등)를 지정합니다.
IMAGE_URL = "images/jeju-beach.jpg"

# 이미지 파일로 부터 질의
answer = multimodal_gemini.stream(IMAGE_URL)

# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
stream_response(answer)